<a href="https://colab.research.google.com/github/HoYoungChun/financial_data_analysis/blob/main/Small_Cap%2BLow_PBR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Small_Cap+Low-PBR

### Basic Setting

In [2]:
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
import urllib.request
# !pip install bs4
# !pip install urllib


In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [5]:
import pandas as pd

csv_url = '/content/drive/MyDrive/Colab/financial_data_analysis/company.csv'

company = pd.read_csv(csv_url, encoding='utf-8')

In [6]:
SNAP_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701'
print(SNAP_URL)

https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701


In [7]:
FS_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=103&stkGb=701'
print(FS_URL)

https://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=103&stkGb=701


In [8]:
sample_code = '035720'

In [9]:
fs = urllib.request.urlopen(FS_URL.format(sample_code)).read()
fs_soup = BeautifulSoup(fs,'html.parser')

In [10]:
snap = urllib.request.urlopen(SNAP_URL.format(sample_code)).read()
snap_soup = BeautifulSoup(snap,'html.parser')

### Market Value(시가총액)

In [11]:
mv_sells = snap_soup.find('div',{'id':"svdMainGrid1"}).find_all('td',{'class':'r'})
market_value = float(mv_sells[8].string.replace(',',''))
market_value

699189.0

### PBR

In [19]:
pbr_cells = snap_soup.find_all('div',{'id':"corp_group2"})[0].find_all('dd')
pbr = float(pbr_cells[-3].string)
pbr

11.06

### Get All Stock code

In [20]:
code_list = company['종목코드']
sample_code_list = code_list[1000:1050]
sample_code_list

1000    207930
1001    214270
1002    189690
1003    213420
1004    206660
1005    060480
1006    206640
1007    208640
1008    193250
1009    204630
1010    200470
1011    200670
1012    208710
1013    189860
1014    067390
1015    080580
1016    208350
1017    160550
1018    124500
1019    142280
1020    187220
1021    208370
1022    207720
1023    208140
1024    149980
1025    084650
1026    200710
1027    200780
1028    207760
1029    196170
1030    206400
1031    178920
1032    140520
1033    173940
1034    205470
1035    200230
1036    194510
1037    204840
1038    196490
1039    204620
1040    205500
1041    205100
1042    191420
1043    192440
1044    182690
1045    143540
1046    203650
1047    041920
1048    194480
1049    203690
Name: 종목코드, dtype: object

In [21]:
def crawler(li):
    # url을 세팅한다.
    SNAP_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701'
    FS_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=103&stkGb=701'

    result={}

    for code in tqdm(li):
        try:
            fs = urllib.request.urlopen(FS_URL.format(code)).read()
            fs_soup = BeautifulSoup(fs,'html.parser')

            snap = urllib.request.urlopen(SNAP_URL.format(code)).read()
            snap_soup = BeautifulSoup(snap,'html.parser')

            #get company name
            name = company[company['종목코드'] == code]['회사명'].values[0]

            #get market value
            mv_sells = snap_soup.find('div',{'id':"svdMainGrid1"}).find_all('td',{'class':'r'})
            market_value = float(mv_sells[8].string.replace(',',''))

            #get pbr
            pbr_cells = snap_soup.find_all('div',{'id':"corp_group2"})[0].find_all('dd')
            pbr = float(pbr_cells[-3].string)

            #insert into result dict
            result[name]=[code, market_value, pbr]

        except(ValueError, AttributeError, IndexError, TypeError):
            pass

    # convert dict into DataFrame
    result_df = pd.DataFrame(result)

    # transposing DataFrame
    result_df = result_df.transpose()

    # set column names
    result_df.columns=['Code','market_value','pbr']
    
    return result_df
        



In [22]:
result_df = crawler(sample_code_list)
result_df

100%|██████████| 50/50 [01:25<00:00,  1.72s/it]


,Code,market_value,pbr
퓨쳐스트림네트웍스,214270,1619,2.18
포시에스,189690,761,1.41
덕산네오룩스,213420,13518,7.01
국일신동,060480,599,1.27
바디텍메드,206640,5273,4.2
썸에이지,208640,4391,18.33
와이제이엠게임즈,193250,1427,2.15
화이브라더스코리아,204630,997,3.11
하이셈,200470,1260,1.87
휴메딕스,200670,3595,2.53


In [23]:
tmp_df = result_df.copy()

### Searching stocks

In [35]:
total_len = tmp_df.shape[0]
#시가총액 하위20%
tmp_df.sort_values(by='market_value', ascending=True).head(int(total_len*0.2))

#pbr 0.2 초과
tmp_df= tmp_df[(tmp_df['pbr'] > 0.2)]

#pbr작은 순으로 정렬
tmp_df.sort_values(by='pbr', ascending=True).head(30)

,Code,market_value,pbr
대창스틸,140520,797,0.89
슈피겐코리아,192440,3506,1.02
디티앤씨,187220,838,1.03
테라셈,182690,305,1.14
아스트,067390,1968,1.23
국일신동,060480,599,1.27
디에이테크놀로지,196490,1413,1.37
포시에스,189690,761,1.41
에프엔씨엔터,173940,909,1.43
지란지교시큐리티,208350,624,1.51
